In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import os
from uuid import uuid4


plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

@dataclass
class MarketMaker:
    """Класс для управления стратегией маркет-мейкинга."""
    spread_bps: float = 10.0
    inventory_limit: int = 1000  
    base_position_decay: float = 0.1
    transaction_cost_bps: float = 1.0
    quote_aggressiveness: float = 2.0
    gamma: float = 0.1
    sigma: float = 0.02
    kappa: float = 1.5
    T: float = 3.0
    tick_size: float = 0.05
    volatility_window: int = 1200

    def __post_init__(self):
        self.position = 0
        self.cash = 0.0
        self.trades = []
        self.current_time = 0.0
        self.price_history = []
        self.sigma = self.sigma * np.sqrt(self.T)

    def calculate_current_volatility(self) -> float:
        """Текущая волатильность на основе последних цен."""
        if len(self.price_history) < self.volatility_window:
            return self.sigma
        prices = np.array(self.price_history[-self.volatility_window:])
        log_returns = np.diff(np.log(prices))
        if len(log_returns) == 0:
            return self.sigma
        volatility = np.std(log_returns) * np.sqrt(252)
        return max(0.001, volatility)

    def get_quotes(self, mid_price: float, market_bid: float = None, market_ask: float = None, use_benchmark: bool = False, use_dynamic_decay: bool = False) -> Tuple[float, float]:
        """Котировки bid и ask."""
        self.price_history.append(mid_price)
        inventory_skew = self.position / self.inventory_limit if self.inventory_limit > 0 else 0

        position_decay = self.base_position_decay
        if use_dynamic_decay:
            current_vol = self.calculate_current_volatility()
            position_decay = self.base_position_decay * (current_vol / self.sigma)
            position_decay = min(max(position_decay, 0.05), 0.5)

        inventory_adjustment = mid_price * (position_decay * inventory_skew / 10000)

        if use_benchmark:
            half_spread_bps = self.spread_bps / 2
            half_spread = mid_price * (half_spread_bps / 10000)
            bid_price = mid_price - half_spread - inventory_adjustment
            ask_price = mid_price + half_spread - inventory_adjustment
        else:
            q = self.position / self.inventory_limit if self.inventory_limit > 0 else 0
            time_to_expiry = self.T - self.current_time
            reservation_price = mid_price - q * self.gamma * (self.sigma ** 2) * time_to_expiry
            spread = (2 / self.gamma) * np.log(1 + (self.gamma / self.kappa))
            bid_price = reservation_price - spread / 2 - inventory_adjustment
            ask_price = reservation_price + spread / 2 - inventory_adjustment

            if market_bid is not None and market_ask is not None and self.quote_aggressiveness > 0:
                improvement = mid_price * (self.quote_aggressiveness / 10000)
                bid_price = max(bid_price, market_bid + improvement)
                ask_price = min(ask_price, market_ask - improvement)

            if market_bid is not None and market_ask is not None:
                bid_price = min(bid_price, market_ask - self.tick_size)
                ask_price = max(ask_price, market_bid + self.tick_size)

        bid_price = np.round(bid_price / self.tick_size) * self.tick_size
        ask_price = np.round(ask_price / self.tick_size) * self.tick_size
        return bid_price, ask_price

    def execute_trade(self, is_buy: bool, price: float, quantity: float, timestamp: float) -> None:
        """Исполняет сделку."""
        position_change = quantity if is_buy else -quantity
        self.position += position_change
        self.cash -= position_change * price
        transaction_cost = price * quantity * (self.transaction_cost_bps / 10000)
        self.cash -= transaction_cost
        self.trades.append({
            'timestamp': timestamp,
            'is_buy': is_buy,
            'price': price,
            'quantity': quantity,
            'position': self.position,
            'cash': self.cash
        })

    def calculate_pnl(self, mid_price: float) -> float:
        """Вычисляет текущую прибыль или убыток."""
        return self.cash + self.position * mid_price

    def update_time(self, current_time: float):
        """Обновляет текущее время в долях дня."""
        self.current_time = current_time / (24 * 3600 * 1e9)

def generate_synthetic_data(
    initial_price: float = 100.0,
    sigma: float = 0.02,
    n_steps: int = 259200,
    dt: float = 1.0 / (24 * 3600),
    spread_bps: float = 10.0
) -> pl.DataFrame:
    """Генерирует синтетические рыночные данные с заданной волатильностью."""
    np.random.seed(42)
    timestamps = np.arange(0, n_steps * 1e9, 1e9)
    mu = 0.0
    prices = np.zeros(n_steps)
    prices[0] = initial_price
    for t in range(1, n_steps):
        dW = np.random.normal(0, np.sqrt(dt))
        prices[t] = prices[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * dW)
    half_spread = prices * (spread_bps / 10000) / 2
    bid_prices = prices - half_spread
    ask_prices = prices + half_spread
    return pl.DataFrame({
        'BTC-USD_timestamp': timestamps,
        'BTC-USD_bid_price_1': bid_prices,
        'BTC-USD_ask_price_1': ask_prices
    })

def run_backtest(
    df: pl.DataFrame,
    instrument_id: str,
    market_maker: MarketMaker,
    execution_prob: float = 0.2,
    latency_steps: int = 0,
    use_benchmark: bool = False,
    use_dynamic_decay: bool = False
) -> Dict:
    """Запускает бэктест стратегии."""
    if df is None or df.is_empty():
        raise ValueError("Данные отсутствуют или пусты.")
    print(f"Обработка {df.height} строк данных")
    timestamps = []
    mid_prices = []
    positions = []
    cash_balances = []
    pnls = []
    bid_quotes = []
    ask_quotes = []
    market_bids = []
    market_asks = []
    valid_rows = 0
    timestamp_col = f'{instrument_id}_timestamp'
    bid_col = f'{instrument_id}_bid_price_1'
    ask_col = f'{instrument_id}_ask_price_1'
    
    for i, row in enumerate(df.iter_rows(named=True)):
        data_idx = max(0, i - latency_steps)
        timestamp = row[timestamp_col]
        market_bid = df[data_idx, bid_col]
        market_ask = df[data_idx, ask_col]
        if np.isnan(market_bid) or np.isnan(market_ask) or market_bid <= 0 or market_ask <= 0:
            continue
        valid_rows += 1
        mid_price = (market_bid + market_ask) / 2
        market_maker.update_time(timestamp)
        bid_price, ask_price = market_maker.get_quotes(mid_price, market_bid, market_ask, use_benchmark, use_dynamic_decay)
        # Динамическая вероятность исполнения с  мягким затуханием
        bid_distance = (market_ask - bid_price) / mid_price if bid_price > 0 else 0
        ask_distance = (ask_price - market_bid) / mid_price if ask_price > 0 else 0
        bid_prob = execution_prob * np.exp(-50 * bid_distance) 
        ask_prob = execution_prob * np.exp(-50 * ask_distance)
        if bid_price > 0 and ask_price > 0:
            if np.random.random() < bid_prob:
                market_maker.execute_trade(True, bid_price, 1, timestamp)
            if np.random.random() < ask_prob:
                market_maker.execute_trade(False, ask_price, 1, timestamp)
        timestamps.append(timestamp)
        mid_prices.append(mid_price)
        positions.append(market_maker.position)
        cash_balances.append(market_maker.cash)
        pnls.append(market_maker.calculate_pnl(mid_price))
        bid_quotes.append(bid_price)
        ask_quotes.append(ask_price)
        market_bids.append(market_bid)
        market_asks.append(market_ask)
    print(f"Обработано {valid_rows} валидных строк, исполнено {len(market_maker.trades)} сделок")
    return {
        'timestamps': timestamps,
        'mid_prices': mid_prices,
        'positions': positions,
        'cash_balances': cash_balances,
        'pnls': pnls,
        'bid_quotes': bid_quotes,
        'ask_quotes': ask_quotes,
        'market_bids': market_bids,
        'market_asks': market_asks,
        'trades': market_maker.trades,
        'instrument_id': instrument_id
    }

def calculate_metrics(pnls: List[float], trades: List[Dict], mid_prices: List[float]) -> Dict:
    """Метрики."""
    pnls = np.array(pnls)
    mid_prices = np.array(mid_prices)
    
   
    if len(pnls) < 2 or np.all(pnls == 0) or np.any(np.isnan(pnls)) or len(mid_prices) < 2 or np.any(np.isnan(mid_prices)):
        print("Предупреждение: Некорректные данные PnL или mid_prices для расчёта метрик")
        return {
            'sharpe_ratio': 0.0,
            'max_drawdown': 0.0,
            'max_position': 0,
            'num_trades': len(trades)
        }

    # Расчёт относительных доходностей
    epsilon = 1e-6  
    pnl_changes = np.diff(pnls)
    avg_abs_pnl_change = np.mean(np.abs(pnl_changes))  
    returns = pnl_changes / (avg_abs_pnl_change + epsilon) 
    
    # Фильтрация выбросов
    q75, q25 = np.percentile(returns, [75, 25])
    iqr = q75 - q25
    returns = returns[(returns >= q25 - 1.5 * iqr) & (returns <= q75 + 1.5 * iqr)]
    returns = returns[~np.isnan(returns) & ~np.isinf(returns)] 
    
    mean_return = np.mean(returns) if len(returns) > 0 else 0.0
    std_return = np.std(returns) if len(returns) > 0 else 1e-6
    
    # Шарп
    if std_return > 1e-6:
        sharpe_ratio = (mean_return / std_return) * np.sqrt(252 / 3) 
    else:
        sharpe_ratio = 0.0
        print("Предупреждение: Нулевая волатильность доходности")

    #Просадки
    equity = pnls
    peak = np.maximum.accumulate(equity)
    max_abs_pnl = np.max(np.abs(pnls)) 
    drawdown = (peak - equity) / (max_abs_pnl + epsilon)  
    max_drawdown = np.max(drawdown) * 100 if np.any(drawdown > 0) else 0.0
    
   
    if max_drawdown > 100:
        print(f"Аномальная просадка: {max_drawdown:.2f}%, min_pnl={np.min(pnls):.2f}, max_pnl={np.max(pnls):.2f}")

    max_position = max(abs(trade['position']) for trade in trades) if trades else 0
    num_trades = len(trades)
    
    return {
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'max_position': max_position,
        'num_trades': num_trades
    }

def plot_results(results: Dict, instrument_id: str, suffix: str = '') -> None:
    """Графики результатов."""
    os.makedirs('plots', exist_ok=True)
    timestamps = pd.to_datetime(results['timestamps'], unit='ns')
    print(f"Сохранение графиков для {instrument_id}{suffix}")
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, results['mid_prices'], label='Mid Price', color='black', alpha=0.5)
    plt.plot(timestamps, results['bid_quotes'], label='Bid Quote', color='green', alpha=0.7)
    plt.plot(timestamps, results['ask_quotes'], label='Ask Quote', color='red', alpha=0.7)
    plt.plot(timestamps, results['market_bids'], label='Market Bid', color='green', alpha=0.3, linestyle='--')
    plt.plot(timestamps, results['market_asks'], label='Market Ask', color='red', alpha=0.3, linestyle='--')
    plt.title(f'Цены и котировки - {instrument_id}{suffix}')
    plt.xlabel('Время')
    plt.ylabel('Цена')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'plots/q2_{instrument_id}_quotes{suffix}.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, results['positions'], label='Позиция', color='blue')
    plt.title(f'Позиция - {instrument_id}{suffix}')
    plt.xlabel('Время')
    plt.ylabel('Позиция')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'plots/q2_{instrument_id}_position{suffix}.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, results['pnls'], label='PnL', color='green')
    plt.title(f'PnL - {instrument_id}{suffix}')
    plt.xlabel('Время')
    plt.ylabel('PnL')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'plots/q2_{instrument_id}_pnl{suffix}.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, results['cash_balances'], label='Кэш', color='orange')
    plt.title(f'Кэш - {instrument_id}{suffix}')
    plt.xlabel('Время')
    plt.ylabel('Кэш')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'plots/q2_{instrument_id}_cash{suffix}.png')
    plt.close()

def analyze_strategy(df: pl.DataFrame, instrument_id: str, sigma: float) -> Tuple[float, int]:
    """Анализирует стратегию с разными настройками и волатильностью."""
    latency_steps_list = [0, 5, 10, 20]
    execution_prob_list = [0.01, 0.05, 0.1]
    transaction_cost_bps_list = [0.5, 1.0, 2.0]
    tick_size_list = [0.01, 0.05, 0.1]
    volatility_scenarios = [('Low', 0.01), ('High', 0.05)]
    results_summary = []
    total_pnl = 0.0
    total_trades = 0
    base_params = {
        'spread_bps': 10.0,
        'inventory_limit': 1000,
        'base_position_decay': 0.1,
        'transaction_cost_bps': 1.0,
        'quote_aggressiveness': 2.0,
        'gamma': 0.8 if sigma == 0.05 else 0.1,  
        'sigma': sigma,
        'kappa': 1.2 if sigma == 0.05 else 1.5, 
        'T': 3.0,
        'tick_size': 0.05,
        'volatility_window': 1200
    }
    for vol_name, sigma in volatility_scenarios:
        print(f"Анализ сценария волатильности: {vol_name}, sigma={sigma}")
        df = generate_synthetic_data(sigma=sigma)
        for use_benchmark in [False, True]:
            for use_dynamic_decay in [False, True]:
                strategy_name = ('Avellaneda-Stoikov' if not use_benchmark else 'Fixed Spread Benchmark') + (' Dynamic Decay' if use_dynamic_decay else '')
                print(f"Тестирование стратегии: {strategy_name}")
                for latency_steps in latency_steps_list:
                    print(f"Тест с задержкой: {latency_steps}")
                    market_maker = MarketMaker(**base_params)
                    results = run_backtest(
                        df=df,
                        instrument_id=instrument_id,
                        market_maker=market_maker,
                        execution_prob=0.05,
                        latency_steps=latency_steps,
                        use_benchmark=use_benchmark,
                        use_dynamic_decay=use_dynamic_decay
                    )
                    final_pnl = results['pnls'][-1]
                    metrics = calculate_metrics(results['pnls'], results['trades'], results['mid_prices'])
                    print(f"Итоговый PnL: {final_pnl:.2f}, Количество сделок: {metrics['num_trades']}, Шарп: {metrics['sharpe_ratio']:.2f}, Просадка: {metrics['max_drawdown']:.2f}%")
                    total_pnl += final_pnl
                    total_trades += metrics['num_trades']
                    results_summary.append({
                        'Strategy': strategy_name,
                        'Volatility': vol_name,
                        'Factor': 'Latency',
                        'Value': latency_steps,
                        'Final PnL': final_pnl,
                        'num_trades': metrics['num_trades'],
                        'sharpe_ratio': metrics['sharpe_ratio'],
                        'max_drawdown': metrics['max_drawdown']
                    })
                    plot_results(results, instrument_id, f'_{strategy_name.replace(" ", "_")}_latency_{latency_steps}_vol_{vol_name}')
                for execution_prob in execution_prob_list:
                    print(f"Тест с вероятностью исполнения: {execution_prob}")
                    market_maker = MarketMaker(**base_params)
                    results = run_backtest(
                        df=df,
                        instrument_id=instrument_id,
                        market_maker=market_maker,
                        execution_prob=execution_prob,
                        latency_steps=0,
                        use_benchmark=use_benchmark,
                        use_dynamic_decay=use_dynamic_decay
                    )
                    final_pnl = results['pnls'][-1]
                    metrics = calculate_metrics(results['pnls'], results['trades'], results['mid_prices'])
                    print(f"Итоговый PnL: {final_pnl:.2f}, Количество сделок: {metrics['num_trades']}, Шарп: {metrics['sharpe_ratio']:.2f}, Просадка: {metrics['max_drawdown']:.2f}%")
                    total_pnl += final_pnl
                    total_trades += metrics['num_trades']
                    results_summary.append({
                        'Strategy': strategy_name,
                        'Volatility': vol_name,
                        'Factor': 'Execution Probability',
                        'Value': execution_prob,
                        'Final PnL': final_pnl,
                        'num_trades': metrics['num_trades'],
                        'sharpe_ratio': metrics['sharpe_ratio'],
                        'max_drawdown': metrics['max_drawdown']
                    })
                    plot_results(results, instrument_id, f'_{strategy_name.replace(" ", "_")}_exec_prob_{execution_prob}_vol_{vol_name}')
                for transaction_cost_bps in transaction_cost_bps_list:
                    print(f"Тест с транзакционными издержками: {transaction_cost_bps}")
                    params = base_params.copy()
                    params['transaction_cost_bps'] = transaction_cost_bps
                    market_maker = MarketMaker(**params)
                    results = run_backtest(
                        df=df,
                        instrument_id=instrument_id,
                        market_maker=market_maker,
                        execution_prob=0.05,
                        latency_steps=0,
                        use_benchmark=use_benchmark,
                        use_dynamic_decay=use_dynamic_decay
                    )
                    final_pnl = results['pnls'][-1]
                    metrics = calculate_metrics(results['pnls'], results['trades'], results['mid_prices'])
                    print(f"Итоговый PnL: {final_pnl:.2f}, Количество сделок: {metrics['num_trades']}, Шарп: {metrics['sharpe_ratio']:.2f}, Просадка: {metrics['max_drawdown']:.2f}%")
                    total_pnl += final_pnl
                    total_trades += metrics['num_trades']
                    results_summary.append({
                        'Strategy': strategy_name,
                        'Volatility': vol_name,
                        'Factor': 'Transaction Cost (bps)',
                        'Value': transaction_cost_bps,
                        'Final PnL': final_pnl,
                        'num_trades': metrics['num_trades'],
                        'sharpe_ratio': metrics['sharpe_ratio'],
                        'max_drawdown': metrics['max_drawdown']
                    })
                    plot_results(results, instrument_id, f'_{strategy_name.replace(" ", "_")}_tx_cost_{transaction_cost_bps}_vol_{vol_name}')
                for tick_size in tick_size_list:
                    print(f"Тест с размером тика: {tick_size}")
                    params = base_params.copy()
                    params['tick_size'] = tick_size
                    market_maker = MarketMaker(**params)
                    results = run_backtest(
                        df=df,
                        instrument_id=instrument_id,
                        market_maker=market_maker,
                        execution_prob=0.05,
                        latency_steps=0,
                        use_benchmark=use_benchmark,
                        use_dynamic_decay=use_dynamic_decay
                    )
                    final_pnl = results['pnls'][-1]
                    metrics = calculate_metrics(results['pnls'], results['trades'], results['mid_prices'])
                    print(f"Итоговый PnL: {final_pnl:.2f}, Количество сделок: {metrics['num_trades']}, Шарп: {metrics['sharpe_ratio']:.2f}, Просадка: {metrics['max_drawdown']:.2f}%")
                    total_pnl += final_pnl
                    total_trades += metrics['num_trades']
                    results_summary.append({
                        'Strategy': strategy_name,
                        'Volatility': vol_name,
                        'Factor': 'Tick Size',
                        'Value': tick_size,
                        'Final PnL': final_pnl,
                        'num_trades': metrics['num_trades'],
                        'sharpe_ratio': metrics['sharpe_ratio'],
                        'max_drawdown': metrics['max_drawdown']
                    })
                    plot_results(results, instrument_id, f'_{strategy_name.replace(" ", "_")}_tick_size_{tick_size}_vol_{vol_name}')

        results_df = pd.DataFrame(results_summary)
        for factor in ['Latency', 'Execution Probability', 'Transaction Cost (bps)', 'Tick Size']:
            for value in results_df[results_df['Factor'] == factor]['Value'].unique():
                subset = results_df[(results_df['Factor'] == factor) & (results_df['Value'] == value) & 
                                    (results_df['Volatility'] == vol_name)]
                print(f"\nСравнение для {factor}={value}, волатильность: {vol_name}")
                print("--------------------------------------------------")
                for _, row in subset.iterrows():
                    print(f"Стратегия: {row['Strategy']}")
                    print(f"  PnL: {row['Final PnL']:.2f}, Сделок: {row['num_trades']}, "
                          f"Шарп: {row['sharpe_ratio']:.2f}, Просадка: {row['max_drawdown']:.2f}%")
                print("--------------------------------------------------")

    print(f"Сохранение результатов анализа в plots/{instrument_id}_analysis_summary.csv")
    results_df = pd.DataFrame(results_summary)
    print(f"Результаты анализа: {len(results_df)} записей")
    print(f"Суммарный PnL для синтетических данных: {total_pnl:.2f}")
    print(f"Общее количество сделок: {total_trades}")
    results_df.to_csv(f'plots/{instrument_id}_analysis_summary.csv', index=False)

    for factor in results_df['Factor'].unique():
        for metric in ['Final PnL', 'sharpe_ratio', 'max_drawdown']:
            print(f"Создание графика: {metric} vs {factor}")
            plt.figure(figsize=(10, 6))
            sns.lineplot(data=results_df[results_df['Factor'] == factor], x='Value', y=metric, hue='Strategy', style='Volatility')
            plt.title(f'{metric} vs {factor} - {instrument_id}')
            plt.xlabel(factor)
            plt.ylabel(metric)
            plt.tight_layout()
            plt.savefig(f'plots/{instrument_id}_{metric.lower().replace(" ", "_")}_vs_{factor.lower().replace(" ", "_")}.png')
            plt.close()

    return total_pnl, total_trades

def main():
    """Запускает анализ стратегии на синтетических данных."""
    print("Запуск анализа стратегии маркет-мейкинга Авелланеда-Стоикова")
    instrument_id = 'BTC-USD'
    total_pnl_all = 0.0
    total_trades_all = 0
    print("Анализ с синтетическими данными...")
    df = generate_synthetic_data()
    synth_pnl, synth_trades = analyze_strategy(df, instrument_id, sigma=0.02)
    total_pnl_all += synth_pnl
    total_trades_all += synth_trades
    print("Анализ завершен успешно.")
    print(f"Общий суммарный PnL по всем тестам: {total_pnl_all:.2f}")
    print(f"Общее количество сделок по всем тестам: {total_trades_all}")
    return 0

if __name__ == "__main__":
    main()

Запуск анализа стратегии маркет-мейкинга Авелланеда-Стоикова
Анализ с синтетическими данными...
Анализ сценария волатильности: Low, sigma=0.01
Тестирование стратегии: Avellaneda-Stoikov
Тест с задержкой: 0
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24777 сделок
Итоговый PnL: 611.84, Количество сделок: 24777, Шарп: 0.09, Просадка: 15.30%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_0_vol_Low
Тест с задержкой: 5
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24745 сделок
Итоговый PnL: 525.86, Количество сделок: 24745, Шарп: 0.15, Просадка: 9.97%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_5_vol_Low
Тест с задержкой: 10
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 25237 сделок
Итоговый PnL: 557.59, Количество сделок: 25237, Шарп: 0.24, Просадка: 6.21%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_10_vol_Low
Тест с задержкой: 20
Обработка 259200 строк данных
Обра

Обработано 259200 валидных строк, исполнено 24664 сделок
Итоговый PnL: 1185.50, Количество сделок: 24664, Шарп: 0.17, Просадка: 4.91%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_latency_20_vol_Low
Тест с вероятностью исполнения: 0.01
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 4996 сделок
Итоговый PnL: 189.30, Количество сделок: 4996, Шарп: 0.19, Просадка: 8.64%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_exec_prob_0.01_vol_Low
Тест с вероятностью исполнения: 0.05
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24499 сделок
Итоговый PnL: 978.85, Количество сделок: 24499, Шарп: 0.11, Просадка: 9.05%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_exec_prob_0.05_vol_Low
Тест с вероятностью исполнения: 0.1
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 48621 сделок
Итоговый PnL: 1866.44, Количество сделок: 48621, Шарп: 0.09, Просадка: 9.46%
Сохранение графиков для BTC-USD_Fixed_

Тестирование стратегии: Avellaneda-Stoikov
Тест с задержкой: 0
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24781 сделок
Итоговый PnL: 1114.93, Количество сделок: 24781, Шарп: 0.02, Просадка: 49.79%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_0_vol_High
Тест с задержкой: 5
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24737 сделок
Итоговый PnL: 717.53, Количество сделок: 24737, Шарп: 0.07, Просадка: 55.37%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_5_vol_High
Тест с задержкой: 10
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 25235 сделок
Итоговый PnL: 785.21, Количество сделок: 25235, Шарп: 0.04, Просадка: 27.53%
Сохранение графиков для BTC-USD_Avellaneda-Stoikov_latency_10_vol_High
Тест с задержкой: 20
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24956 сделок
Итоговый PnL: 251.46, Количество сделок: 24956, Шарп: 0.02, Просадка: 92.77%
Сохране

Тест с задержкой: 10
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24694 сделок
Итоговый PnL: 1111.37, Количество сделок: 24694, Шарп: 0.02, Просадка: 46.23%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_latency_10_vol_High
Тест с задержкой: 20
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24670 сделок
Итоговый PnL: 1983.23, Количество сделок: 24670, Шарп: 0.03, Просадка: 25.39%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_latency_20_vol_High
Тест с вероятностью исполнения: 0.01
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 5001 сделок
Итоговый PnL: 137.07, Количество сделок: 5001, Шарп: 0.05, Просадка: 55.57%
Сохранение графиков для BTC-USD_Fixed_Spread_Benchmark_exec_prob_0.01_vol_High
Тест с вероятностью исполнения: 0.05
Обработка 259200 строк данных
Обработано 259200 валидных строк, исполнено 24488 сделок
Итоговый PnL: 1024.57, Количество сделок: 24488, Шарп: 0.02, Просадка: 60.09

Создание графика: sharpe_ratio vs Latency
Создание графика: max_drawdown vs Latency
Создание графика: Final PnL vs Execution Probability
Создание графика: sharpe_ratio vs Execution Probability
Создание графика: max_drawdown vs Execution Probability
Создание графика: Final PnL vs Transaction Cost (bps)
Создание графика: sharpe_ratio vs Transaction Cost (bps)
Создание графика: max_drawdown vs Transaction Cost (bps)
Создание графика: Final PnL vs Tick Size
Создание графика: sharpe_ratio vs Tick Size
Создание графика: max_drawdown vs Tick Size
Анализ завершен успешно.
Общий суммарный PnL по всем тестам: 79769.12
Общее количество сделок по всем тестам: 2614416
